In [328]:
def PLOT(img, anno, bbox_dis = False):
    fig, ax = plt.subplots()
    cmp = ['#ff0000', '#00ff00', '#0000ff', '#ffff00', '#00ffff', '#ff00ff', '#ff8000', '#80ff00']
    skeleton = [[15,13],[13,11],[16,14],[14,12],[11,12],[5,11],[6,12],[5,6],[5,7],[6,8],[7,9],[8,10],[1,2],[0,1],[0,2],[1,3],[2,4],[3,5],[4,6]]
    ax.imshow(img)
    kypt = anno['keypoints']
    anno_x = []
    anno_y = []
    anno_w = []
    if len(kypt[0]) < 3:
        for k in kypt:
            anno_x.append(k[0])
            anno_y.append(k[1])    
        anno_w = anno['kweights']
    else:
        for k in range(int(len(kypt[0])/3)):
            anno_x.append(kypt[0][k*3])
            anno_y.append(kypt[0][k*3+1])
            anno_w.append(kypt[0][k*3+2])
    ax.scatter(anno_x, anno_y, marker = '.', color = cmp[1])
    for i in skeleton:
        #if anno_x[i[0]] != 0 and anno_x[i[1]] != 0 and anno_y[i[0]] != 0 and anno_y[i[1]] != 0:
        if anno_w[i[0]] != 0 and anno_w[i[1]] != 0:
            x = []
            y = []
            x.append(anno_x[i[0]])
            x.append(anno_x[i[1]])
            y.append(anno_y[i[0]])
            y.append(anno_y[i[1]])
            ax.plot(x, y , linestyle = '-', color = cmp[1])
    if bbox_dis:
        bbox = []
        codes = []
        vertices = []
        codes = [Path.MOVETO] + [Path.LINETO]*3 + [Path.CLOSEPOLY]
        bbox.append(anno['bbox'][0][0])
        bbox.append(anno['bbox'][0][1])
        bbox.append(anno['bbox'][0][0] + anno['bbox'][0][2])
        bbox.append(anno['bbox'][0][1] + anno['bbox'][0][3])
        vertices = [(bbox[0], bbox[1]), (bbox[0], bbox[3]), (bbox[2], bbox[3]), (bbox[2], bbox[1]), (0, 0)]
        path = Path(vertices, codes)
        pathpatch = PathPatch(path, facecolor ='none', edgecolor = cmp[1])
        ax.add_patch(pathpatch)
    plt.show()
    #fig.savefig('./output.jpg')
    return

In [ ]:
def PLOT_HEATMAP(heatmap):
    label = ["nose","left_eye","right_eye","left_ear", 
             "right_ear","left_shoulder","right_shoulder",
             "left_elbow","right_elbow","left_wrist","right_wrist",
             "left_hip","right_hip","left_knee","right_knee","left_ankle","right_ankle"]
    for i in range(17):
        plt.subplot(5, 4, i+1)
        plt.imshow(heatmap[i])
        plt.title(label[i])
    plt.tight_layout()
    plt.show()
    return

In [ ]:
def gaussian_heatmap(anno, shape, sigma=8):
    kypt = anno['keypoints']
    heatmap = []
    for i in range(len(kypt)):
        if anno['weights'][i] != 0: 
            coords = kypt[i]
            x, y = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
            dists = np.sqrt((x[:, :, np.newaxis] - coords[0])**2 +
                            (y[:, :, np.newaxis] - coords[1])**2)
            gaussians = np.exp(-dists**2 / (2 * sigma**2))
            tmp = np.sum(gaussians, axis=2)
            tmp /= np.max(tmp)
            heatmap.append(tmp)
        else:
            heatmap.append(np.zeros(shape))
    heatmap = np.array(heatmap)
    return heatmap

In [1]:
%load_ext memory_profiler

In [294]:
import os
import cv2
import time
import json
import psutil
import numpy as np
from PIL import Image
import albumentations as A
from decimal import Decimal
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from pycocotools.coco import COCO
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as F
from albumentations.pytorch.transforms import ToTensorV2
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = [10, 10]

In [320]:
class COCO2017Keypoint(Dataset):
    def __init__(self, PATH, transforms = None):
        super(COCO2017Keypoint, self).__init__()
        with open(PATH,'r') as f:
            self.PATH = json.load(f)
        self.api = COCO(annotation_file = os.path.join(self.PATH['ROOT'], self.PATH['ValAnn']))
        self.ids = list(self.api.anns.keys())
        self.transforms = transforms
    
    def __getitem__(self, index):
        anno_tmp = self.api.loadAnns(ids = self.ids[index])[0]
        anno = {'keypoints' : [], 'kweights' : []}
        bbox = []
        bbox.append(int(anno_tmp['bbox'][0]))
        bbox.append(int(anno_tmp['bbox'][1]))
        bbox.append(int(anno_tmp['bbox'][0] + anno_tmp['bbox'][2]))
        bbox.append(int(anno_tmp['bbox'][1] + anno_tmp['bbox'][3]))
        #for i in range(int(len(anno_tmp['keypoints'])/3)):
        num_of_keypoints = int(len(anno_tmp['keypoints'])/3)
        for i in range(num_of_keypoints):
            tmp = [anno_tmp['keypoints'][i*3],anno_tmp['keypoints'][i*3+1], anno_tmp['keypoints'][i*3+2]]
            if tmp[2] == 0:                   
                anno['keypoints'].append([0, 0])
                anno['kweights'].append(0)
            elif tmp[0] > bbox[0] and tmp[0] < bbox[2] and tmp[1] > bbox[1] and tmp[1] < bbox[3]:
                tmp[0] -= bbox[0]
                tmp[1] -= bbox[1]
                anno['keypoints'].append([tmp[0], tmp[1]])
                anno['kweights'].append(tmp[2])
            else:                
                anno['keypoints'].append([0, 0])
                anno['kweights'].append(0)
        img = Image.open(os.path.join(self.PATH['ROOT'],
                                      self.PATH['ValImg'], 
                                      self.api.loadImgs(ids = anno_tmp['image_id'])[0]['file_name']))
        img = np.array(img.crop((bbox[0], bbox[1], bbox[2], bbox[3])))
        if self.transforms is not None:
            transformed = self.transforms(image = img, keypoints = anno['keypoints'])
            img = transformed['image']
            anno['keypoints'] = transformed['keypoints']
        return img, anno
    
    def __len__(self):
        return len(self.ids)

In [321]:
transforms = A.Compose([A.RandomBrightnessContrast(p = 0.5), 
                        A.SafeRotate(interpolation = cv2.INTER_LINEAR, border_mode = cv2.BORDER_CONSTANT, value = 0, p = 0.5),
                        A.Resize(height=224,
                                    width=224,
                                    interpolation = cv2.INTER_LINEAR),
                        ToTensorV2()
                        A.Normalize()],
                       keypoint_params = A.KeypointParams(format = 'xy', remove_invisible = False))
data = COCO2017Keypoint(PATH = 'PATH.json', transforms = transforms)
val_loader = DataLoader(data, batch_size = k, shuffle = False, num_workers = 2)

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!


In [ ]:
for k in range(1,128):    
    for idx, img in enumerate(val_loader):
        imF.to_pil_image(img[0][0])
        # = plt.imshow(im)
        #plt.show()
        break
    print(k)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
